In [2]:
# This code creates geojson files of the DEM and optical labels that are within the tiles

# Further unionizes optical training and predictions labels so that there are no overlapping labels with the same year

# It also calculates Area, Solidity, and Circularity of the labels

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import rasterio as rio
from shapely.geometry import Polygon
from mpl_toolkits.basemap import Basemap  
import matplotlib.pyplot as plt
from shapely.ops import unary_union
import math

In [4]:
from shapely.geometry import MultiPolygon
import geopandas as gpd
from shapely.ops import unary_union

# Unionize the touching geometries with same year to create single polygons 
def unionize_geometries_by_year(geo_df, year_column='year', geometry_column='geometry'):
    union_results = []
    years = geo_df[year_column].unique()

    for year in years:
        subset = geo_df[geo_df[year_column] == year]
        
        # Find touching groups
        touching_groups = [list(group[geometry_column]) for _, group in subset.groupby(subset.touches(subset))]
        
        # Unionize touching groups
        unionized_touching_group = unary_union(touching_groups)
        
        # Find polygons that don't touch any other polygon within the same year
        non_touching_polygons = subset[~subset.index.isin(subset.touches(subset).index)]
        
        # Add non-touching polygons to the union results only if they exist
        if not non_touching_polygons.empty:
            non_touching_geometries = list(non_touching_polygons[geometry_column])
            for geometry in non_touching_geometries:
                union_results.append({year_column: year, geometry_column: geometry})
        
        # Add unionized touching group to the union results
        if unionized_touching_group.is_empty == False:
            union_results.append({year_column: year, geometry_column: unionized_touching_group})

    # Create a GeoDataFrame from the results list
    union_df = gpd.GeoDataFrame(union_results, crs=geo_df.crs)
    union_df = union_df.explode()
    
    return union_df

# Area
def get_area(labels):
    labels['area'] = labels['geometry'].area
    return labels

# Circularity
def get_circularity(geometry):
    perimeter = geometry.length
    area = geometry.area
    circularity = (4 * math.pi * area) / (perimeter ** 2)
    return circularity

#Solidity
def get_solidity(geometry):
    convexhull = geometry.convex_hull
    convexhull = convexhull.area
    area = geometry.area
    solidity = area / convexhull
    return solidity

In [5]:
def process_region(region_name, footprint_file):
    # Optical Label Files
    optical_training_labels = gpd.read_file('labels\\optical_labels\\optical_training_labels.geojson')
    optical_prediction_labels = gpd.read_file('labels\\optical_labels\\optical_predictions.gpkg')

    # Read DEM label files
    DEM_labels = gpd.read_file(f'labels\\DEM_labels\\{region_name}_DEM_labels_rasterized.geojson')

    # Read Tiles and Footprints
    tiles = gpd.read_file(f'Results\\{region_name}\\Footprints\\{region_name}_area.geojson')
    footprint = gpd.read_file(f'boundries\\footprints\\CRS_adjusted\\{footprint_file}')

    # Spatial join Footprint with Tiles
    fp = gpd.sjoin(footprint, tiles, predicate='overlaps', how='inner')
    fp = fp.drop_duplicates(subset='image_id')
    fp = fp.drop(columns=['index_right', 'datasource', 'subset', 'fid', 'tile_name'])
    fp['region'] = region_name

    # Footprint year
    fp['image_year'] = pd.to_datetime(fp['image_date']).dt.year

    years = fp['image_year'].unique()

    for year in years:
        fp_year = fp[fp['image_year'] == year]
        fp_year_union = unary_union(fp_year['geometry'])
        fp_year = gpd.GeoDataFrame(geometry=[fp_year_union], crs=fp.crs)
        fp_year.to_file(f'Results\\{region_name}\\Footprints\\footprint_{region_name}_' + str(year) + '.geojson')

    # Spatial join for Optical and DEM labels
    OP_lab = gpd.sjoin(optical_prediction_labels, tiles, predicate='within',how='inner')
    OT_lab = gpd.sjoin(optical_training_labels, tiles, predicate='within', how='inner')
    DEM_labels = gpd.sjoin(DEM_labels, tiles, predicate='within', how='inner')

    # Get Rid of Unnecessary Columns
    DEM_labels = DEM_labels.drop(columns=['index_right', 'tile_name'])
    OT_lab = OT_lab.drop(columns=['index_right', 'site', 'id_ds', 'label', 'datasource', 'fid', 'subset', 'target', 'author', 'id', 'image_id', 'layer','tile_name'])    
    OP_lab = OP_lab.drop(columns=['index_right', 'satellite', 'image_id', 'DN', 'tile_id', 'take_id', 'tile_name'])

    # Processing and saving labels with metrics
    OT_lab['year'] = pd.to_datetime(OT_lab['image_date']).dt.year
    OT_lab.geometry = OT_lab.geometry.buffer(0)
    OT_lab = unionize_geometries_by_year(OT_lab)
    OP_lab = unionize_geometries_by_year(OP_lab)
    OP_lab['region'] = region_name
    OT_lab['region'] = region_name
    DEM_labels['region'] = region_name

    Labels = [DEM_labels, OT_lab, OP_lab]

    for label in Labels:
        label = get_area(label)
        label['circularity'] = label['geometry'].apply(get_circularity)
        label['solidity'] = label['geometry'].apply(get_solidity)

    # Save files with metrics
    OT_lab.to_file(f'labels\\labels_w_metrics\\{region_name}_OT_labels_unionized_SimpleMetrics.geojson', driver='GeoJSON')
    OP_lab.to_file(f'labels\\labels_w_metrics\\{region_name}_OP_labels_unionized_SimpleMetrics.geojson', driver='GeoJSON')
    DEM_labels.to_file(f'labels\\labels_w_metrics\\{region_name}_DEM_labels_rasterized_SimpleMetrics.geojson', driver='GeoJSON')


In [6]:
# Define regions and corresponding footprints
regions_footprints = {
    'herschel': 'footprint001.geojson',
    'kolguev': 'footprint001.geojson',
    'peel': 'footprint002.geojson',
    'gydan': 'footprint003.geojson'
}

# Process each region
for region, footprint_file in regions_footprints.items():
    process_region(region, footprint_file)

C:\Users\Noe\AppData\Local\Temp\ipykernel_28584\1084091739.py:34: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  union_df = union_df.explode()
C:\Users\Noe\AppData\Local\Temp\ipykernel_28584\1084091739.py:34: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  union_df = union_df.explode()
C:\Users\Noe\AppData\Local\Temp\ipykernel_28584\1084091739.py:34: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  union_df = union_df.explode()
C:\Users\Noe\AppData\Local\Temp\ipykernel_2